In [ ]:
import os
os.chdir(r'<<FILLME/backend>>')

In [ ]:
import cv2
import torch 
import faiss

# Local imports
from siamese.siamese_pt.model import create_model
from siamese.siamese_pt.dataset import common_transforms
from siamese.test_index import read_index, query_index, display_query_results
from siamese.utils import torch_to_cv2, denormalize
from siamese.siamese_pt.dataset import common_transforms
import siamese.config as config


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = create_model()
model.load_state_dict(torch.load("densenet121_model.pth"))
model.eval()

query_paths = list(config.QUERY_DATASET.rglob("**/*.[jp][pn]g"))
print(f"There are {len(query_paths)} images for querying.")

index = read_index()
for impath in query_paths:
    image = cv2.imread(impath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = common_transforms(image=image)["image"]
    image = image.to(device, dtype=torch.float32)
    image = image.unsqueeze(0)
    embedding = model(image)
    embedding = embedding.detach().cpu().numpy()
    faiss.normalize_L2(embedding)

    indices, distances = query_index(
        embedding, index, config.INDEX_TYPE, config.N_RESULTS
    )

    image = denormalize(image)
    image = torch_to_cv2(image)
    display_query_results(image, distances, indices)